In [49]:
from aicspylibczi import CziFile
import numpy as np
import skimage as sk
import pandas as pd
import os
from glob import glob
import napari
from cellpose import models, io, utils
#import pyclesperanto as cle


In [38]:
def normalize_images_CPU(input):
    equalized_stack = np.zeros_like(input)
    num_slices = input.shape[1]
    num_channels = input.shape[0]
    if num_channels > 1:
        for channel in range(num_channels):
            channel_stack = input[channel,...]
            mean_intensity_stack = sk.filters.threshold_mean(channel_stack)
            corrected_stack = np.astype(np.stack([channel_stack[z,...]*(sk.filters.threshold_mean(channel_stack[z,...])/mean_intensity_stack) for z in range(num_slices)], axis=0),np.uint16)
            background_subtracted = np.astype(sk.morphology.white_tophat(corrected_stack),np.uint16)
            np.copyto(equalized_stack[channel,...],background_subtracted)
    else:
        mean_intensity_stack = sk.filters.threshold_mean(input)
        corrected_stack = np.astype(np.stack([input[z,...]*(sk.filters.threshold_mean(input[z,...])/mean_intensity_stack) for z in range(num_slices)], axis=0),np.uint16)
        background_subtracted = sk.morphology.white_tophat(corrected_stack)
        np.copyto(equalized_stack,background_subtracted)
    return equalized_stack           


In [3]:
file_paths = sorted(glob("E:\Fondufe-Mittendorf_Lab\OIC-150_Images\*.czi"))
czi_files = list(map(CziFile,file_paths))

In [50]:
test_img = czi_files[0]

In [51]:
img = [test_img.read_image(S=i)[0] for i in range(test_img.size[2])]

In [88]:
img = np.squeeze(img)
img.shape

(3, 4, 24, 1012, 1012)

In [95]:
test = img[0]
test.shape

(4, 24, 1012, 1012)

In [99]:
gauss = sk.filters.gaussian(test[-1,...])

In [100]:
io.logger_setup()
model = models.Cellpose(gpu=True,model_type='cyto3') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _, _ = model.eval(gauss, diameter=100, channels=channels, z_axis=0,stitch_threshold=0.3,flow_threshold=0.4,cellprob_threshold=2.0, min_size=100)

creating new log file
2025-06-20 12:20:06,372 [INFO] WRITING LOG OUTPUT TO C:\Users\kristin.gallik\.cellpose\run.log
2025-06-20 12:20:06,373 [INFO] 
cellpose version: 	3.1.1.2 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.7.0+cu118
2025-06-20 12:20:06,375 [INFO] ** TORCH CUDA version installed and working. **
2025-06-20 12:20:06,376 [INFO] >>>> using GPU (CUDA)
2025-06-20 12:20:06,377 [INFO] >> cyto3 << model set to be used
2025-06-20 12:20:06,490 [INFO] >>>> loading model C:\Users\kristin.gallik\.cellpose\models\cyto3
2025-06-20 12:20:06,772 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-06-20 12:20:06,775 [INFO] channels set to [[0, 0]]
2025-06-20 12:20:06,775 [INFO] ~~~ FINDING MASKS ~~~
2025-06-20 12:20:06,776 [INFO] multi-stack tiff read in as having 24 planes 1 channels
2025-06-20 12:20:08,039 [INFO] 100%|##########| 12/12 [00:00<00:00, 12.62it/s]
2025-06-20 12:20:08,235 [INFO] network run in 1.15s
2025-06-20 12:20:

100%|██████████| 23/23 [00:00<00:00, 124.51it/s]


2025-06-20 12:20:22,262 [INFO] masks created in 14.03s
2025-06-20 12:20:24,255 [INFO] >>>> TOTAL TIME 17.48 sec


In [101]:
voxel = [0.25,0.13,0.13]
viewer = napari.view_image(gauss, scale=voxel)
viewer.add_labels(masks,scale=voxel)

<Labels layer 'masks' at 0x16d7b7ffbe0>